<a href="https://colab.research.google.com/github/yukinaga/automl/blob/main/section_1/01_simple_automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# シンプルなAutoML
ライブラリPyCaretを使い、AutoMLによる機械学習を実装します。  
顧客情報から、その顧客がデフォルト（債務不履行）するかどうかを判定します。  

## PyCaretのインストール
AutoMLをサポートするライブラリ、PyCaretをバージョンを指定してインストールします。

In [ ]:
!pip install pycaret==2.3.6
!pip install pandas-profiling==3.1.0

## Google Colabの設定
Google Colab環境でPyCaretのインタラクティブな要素を表示するためには、以下のコードを実行する必要があります。

In [ ]:
from pycaret.utils import enable_colab

enable_colab()

## データセットの読み込み
今回は、UCIのDefault of Credit Card Clients Datasetというデータセットを使います。  
このデータセットには、2005年4月から2005年9月までの、台湾のクレジットカード顧客情報が含まれます。  
顧客情報には、デフォルト（債務不履行）したかどうか、人口統計学的要因、信用データ、支払い履歴、および請求書の情報があります。  
サンプル数は24000で特徴の数は25です。  
以下は各特徴の簡単な説明です。  

* ID： 各顧客のID
* LIMIT_BAL: ニュー台湾ドル建ての与信額
* SEX: 性別(1=男性、2=女性)
* EDUCATION： 教育（1＝大学院、2＝大学、3＝高校、4＝その他、5＝不明、6＝不明)
* MARRIAGE: 結婚の有無（1＝既婚、2＝独身、3＝その他)
* AGE: 年齢（単位：歳)
* PAY_0～PAY_6： nヶ月前の返済状況（-1=正規の支払い、1=1ヶ月間の支払い遅延、2=2ヶ月間の支払い遅延、...)
* BILL_AMT1～BILL_AMT6： nヶ月前までの請求明細の金額
* PAY_AMT1 to PAY_AMT6: nヶ月前までの支払額
* default: 債務不履行したかどうか (1=yes, 0=no) 

https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

In [ ]:
from pycaret.datasets import get_data

dataset = get_data("credit")  # データの取得
dataset.shape  # データの形状

## 環境の設定
PyCaretの環境を設定します。  
setup関数はPyCaretの環境を初期化しますが、PyCaretの他の関数を実行する前に呼び出す必要があります。  

以下のコードを実行すると、setup関数が実行されてすべての特徴のデータ型が自動的に推定されます。  
この際に、必ずしも正しく推論されるとは限らないことにご注意ください。     
データの型に問題が無ければ、空白を入力することで設定を完了することができます。  

In [ ]:
from pycaret.classification import setup

clf = setup(data=dataset, target="default", session_id=123)  # 環境の初期化

## モデルの比較
様々なモデルを比較して、性能を評価します。  
compare_models関数は、ライブラリ内のすべてのモデルを使って訓練を行い、スコアを評価します。  

In [ ]:
from pycaret.classification import compare_models

best_model = compare_models()  # 全てのモデルを訓練し、評価する

わずか1行のコードで、15以上の機械学習モデルを訓練し、評価することができました。  
  
最もスコアの良いモデルの概要を表示します。


In [ ]:
print(best_model)

## モデルの作成
models関数により、全ての使用可能な機械学習モデルを確認することができます。

In [ ]:
from pycaret.classification import models

models()  # 機械学習モデルの一覧

create_model関数は、「交差検証」を用いて個別のモデルの訓練と評価を行います。  
デフォルトでは、データを10に分割し10通りのテストデータ/訓練データの組み合わせを試します。     
今回は、ツリー状の構造を使ってデータを分類する「決定木」のモデルを作成します。  

In [ ]:
from pycaret.classification import create_model

dt = create_model("dt")  # 決定木のモデルを作成

訓練済みモデルの概要を表示します。  

In [ ]:
print(dt)

## ハイパーパラメータの調整
create_model関数は、デフォルトのハイパーパラメータを使ってモデルを訓練します。  
ハイパーパラメータを調整するには、tune_model関数を使用します。  

In [ ]:
from pycaret.classification import tune_model

tuned_dt = tune_model(dt)  # ハイパーパラメータの調整

ハイパーパラメータを調整済みの、訓練済みモデルの概要を表示します。  

In [ ]:
print(tuned_dt)

## モデルを評価する
plot_model関数により、様々な側面からモデルを分析することができます。  
15種類のプロットが利用可能です。  

https://pycaret.org/plot-model/

以下のコードは、各特徴の重要度をグラフにプロットします。  

In [ ]:
from pycaret.classification import plot_model

plot_model(tuned_dt, plot="feature")  # 各特徴の重要度をプロット